In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from itertools import combinations
from queue import PriorityQueue
import os

/home/lehniise/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/lehniise/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/home/lehniise/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/lehniise/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


## MLP module

In [2]:
class MLP(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)  
        )
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

In [3]:
class MLP_singleton(nn.Module): #embed singletons
    def __init__(self, input_dim = 2, hidden_dim = 3, output_dim=8):
        super(MLP_singleton, self).__init__()
        self.node = MLP(input_dim, hidden_dim, output_dim)
        #nn.ReLU() 
        
    def forward(self, x): #input here is a point
        x = self.node(x) #this is mapping the embedding of leaf to higher dimension
        return x

In [4]:
class MLP_init(nn.Module):
    def __init__(self, input_dim = 2, hidden_dim = 3, output_dim = 8):
        super(MLP_init, self).__init__()
        self.init = MLP(input_dim, hidden_dim, output_dim)
        #nn.ReLU() 
        
    def forward(self, x, y): #input here is pair of singletons
        z = self.init(x+y) #computing pairwise embeddings
        return z
        

In [5]:
class MLP_update(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP_update, self).__init__()
        self.update = MLP(input_dim, hidden_dim, output_dim)
        #nn.ReLU() 
        
    def forward(self, x, y, z): #input here is a list of 3 clusters/singletons
        final = np.concatenate(x+y, z)
        final = self.update(final)
        return final

In [6]:
class pairwise_ranker(nn.Module):
    def __init__(self, input_dim):
        super(pairwise_ranker, self).__init__()
        self.w = nn.Parameter(torch.rand(input_dim))
       
    def forward(self, x, y):
       z = x.embedding - y.embedding
       z = torch.tanh(x*z)
       return z

In [7]:
#Embedd user data
class cluster:
    def __init__(self, label, data, embedding):
        self.label = label
        self.data = data
        self.embedding = embedding
        self.children = []
        
    def add_child(self, child_cluster):
        if isinstance(child_cluster, cluster):
             self.children.append(child_cluster)
        else:
             raise TypeError("Child is not a cluster.")
    
    def __lt__(self, other):
        rank = pairwise_ranker(self, other)
        return rank > 0

In [8]:
def main(data):
    def generate_rand_label(length=3):
        return os.urandom(length).hex()

    def get_pairwise_sums(singleton1, singleton2, embeddings): ## Getting pairwise sums 
        embed1 = embeddings.get(singleton1)
        embed2 = embeddings.get(singleton2)
        label = singleton1+singleton2
        data = embed1.data+embed2.data
    
        temp_embed = MLP_init(embed1.data, embed2.data)
    
        temp_embed_object = cluster(label, data, temp_embed)
        temp_embed_object.add_child(embed1)
        temp_embed_object.add_child(embed2)    
    
        embeddings[label] = temp_embed_object
        merges.put(temp_embed_object)
    
    embeddings = {} #consists of singletons [A], [B] and clusters i.e [AB]
    merges = PriorityQueue()
    #MLP_singletons
    for point in data: #assuming data is a list of vectors
        embedding = MLP_singleton(point)
        label = generate_rand_label()
        temp_embed_object = cluster(label, point, embedding)
        embeddings[label] = temp_embed_object #adding data_point object to dictionary
        
    print("Finished running MLP_singleton")

    #MLP_init
    for a, b in combinations(list(embeddings.keys()), 2):
        get_pairwise_sums(a,b, embeddings)
        
    print("Finished running MLP_init")
        
    #MLP_update
   
    new_candidates = []
    
    for merge in merges: 
        embed_1 = embeddings.get(merge[0])
        embed_2 = embeddings.get(merge[1])
        dissected_1 = embed_1.label
        dissected_2 = embed_2.label
        dissected = set(list(dissected_1)).union(set(list(dissected_2))) ## what am I doing here?
    
    for key in embeddings.keys():
        key_dissect = set(list(key))
        common_elements = dissected.intersection(key_disect)
        if len(common_elements) == 0:
            new_candidates = new_candidates.append(key)

    for candidate in new_candidates: #compute embedding of new cluster to every other cluster
            merge_cluster_label = dissected_1+dissected_2
            clust_1 = embeddings.get(dissected_1 + candidate)
            clust_1 = clust_1.data
            clust_2 = embeddings.get(dissected_2 + candidate)
            clust_2 = clust_2.data
            clust_3 = embeddings.get(merge_cluster_label)
            clust_3 = clust_3.data
   
            new_embedding = MLP_update(clust_1, clust_2, clust_3)
            embeddings[merge_cluster_label+candidate] = cluster(merge_cluster_label, )
    print("Finished running MLP_update")     
    return 1
            
'''   
    for merge in merges: #here we begin going through the given merges and performing updates
        dissected_1 = set(list(merge[0])) #cluster 1 of merge
        dissected_2 = set(list(merge[1])) #cluster 2 of merge
        dissected = dissected_1.union(dissected_2) #get rid of 
        
        for key in embeddings.keys():
            key_dissect = set(list(key))
            common_elements = dissected.intersection(key_disect)
            if len(common_elements) == 0:
                new_candidates = new_candidates.append(key)
                
        for candidate in new_candidates: #compute embedding of new cluster to every other cluster
            get_preexisting = embeddings[merge[0]+candidate] + embeddings[merge[1]+candidate] #these should be summed element wise
            new_embedding = np.concatenate((get_preexisting, embeddings[merge[0]+merge[1]])) #for column concatenation set axis=1
            
        final = MLP_update(new_embedding)
        embeddings['#go back and track name of cluster here'] = final #need to update data structure used here
'''           

if __name__ == "__main__":
    main(([1,2]))

Finished running MLP_singleton
Finished running MLP_init


TypeError: 'PriorityQueue' object is not iterable

In [ ]:
#NOTES

'''
- test basic functionality on synthetic datasets
- implement tree object
'''